In [1]:
%pylab
%matplotlib inline
rc('text', usetex = True)

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


In [280]:
import pyfits as fits
from astropy.utils.data import download_file
import pandas as pd
from lmfit import Parameters, minimize
from numpy.linalg import lstsq, solve

In [281]:
### Download data


### Cepheid table
cepheid_table = download_file(
        'http://iopscience.iop.org/0004-637X/730/2/119/suppdata/apj383673t2_mrt.txt', 
        cache = True)
cepheids = pd.read_csv(cepheid_table,
                       delim_whitespace = True,
                       skiprows = 39,
                       names = (['Field', 'RAdeg', 
                                 'DEdeg', 'ID', 
                                 'Period', 'VtoI', 
                                 'F160Wmag', 'e_F160Wmag',
                                 'Offset', 'Bias', 
                                 'IMrms', 'ObyH', 'Flag']
                               )
                      )

cepheids=cepheids.fillna(value = '-')
### SNe table
Sne_table = download_file(
        'http://iopscience.iop.org/0004-637X/730/2/119/suppdata/apj383673t3_ascii.txt',
        cache = True)
Sne = pd.read_csv(Sne_table, 
                   
                  delim_whitespace=True, 
                  skiprows = [0,1,2,3,4,13,15],
                  names = (['Host', 'junk','Sn1A',
                            'Filters', 'm0_viPlus5a_v',
                            'sigma', 'DeltaMu_0','e_DeltaMu_0',
                            'mu_0_Best', 'e_mu_0_Best'
                          ])
                 )
Sne.loc[:,'e_DeltaMu_0'] = (Sne.loc[:,'e_DeltaMu_0'].apply(str).str.replace('\(|\)','')).astype('float')
Sne.loc[:,'e_mu_0_Best'] = (Sne.loc[:,'e_mu_0_Best'].apply(str).str.replace('\(|\)','')).astype('float')


maser_distance = {'mu':7.2, 'e_mu':0.32} 
maser_distance = pd.DataFrame(data = maser_distance, index = arange(1))

In [282]:
set(cepheids.Field)

{'n1309',
 'n3021',
 'n3370',
 'n3982',
 'n4038',
 'n4258',
 'n4536',
 'n4639',
 'n5584'}

In [283]:
cepheids = cepheids.loc[cepheids.Flag == '-',:]
cepheids = cepheids.reset_index(drop = True)
y = cepheids.F160Wmag -0.411 * cepheids.VtoI
y = list(y)
print len(y)
y += list(Sne.m0_viPlus5a_v - 0.698 * 5)
y = array(y)
y.size


yerr = cepheids.e_F160Wmag.tolist()
yerr += Sne.sigma.tolist()
yerr = array(yerr)
Cinv = diag(1.0/yerr**2)
print Cinv

444
[[ 10.40582726   0.           0.         ...,   0.           0.           0.        ]
 [  0.           5.66893424   0.         ...,   0.           0.           0.        ]
 [  0.           0.           5.16528926 ...,   0.           0.           0.        ]
 ..., 
 [  0.           0.           0.         ...,  94.25959091   0.           0.        ]
 [  0.           0.           0.         ...,   0.          67.18624026
    0.        ]
 [  0.           0.           0.         ...,   0.           0.          53.2793436 ]]


In [284]:
set(cepheids.Field)

{'n1309',
 'n3021',
 'n3370',
 'n3982',
 'n4038',
 'n4258',
 'n4536',
 'n4639',
 'n5584'}

In [306]:
paramnames = []
fields = list(set(cepheids.Field))
fields.remove('n4258')
L = zeros([y.size, 12])
L[:444,-4] = log10(cepheids.Period) ## PL-period slope
L[:444,-3] = cepheids.ObyH - mean(cepheids.ObyH) ## PL-metallicity slope
L[:444, -2] = 1 ## PL-intercept
L[444:,-1] = 1 ## M^0_4258
for i, field in enumerate(fields):
    print field
    if field != 'n4258':
        cephindex = cepheids.loc[cepheids.Field == field, :].index.tolist()
        L[cephindex,i] = 1
        snindex = Sne.loc[Sne.Host == field,:].index.tolist()
        snindex = [x + 444 for x in snindex]
        L[snindex,i] = 1
        paramnames.append(field)
paramnames.append('Per')
paramnames.append('Met')
paramnames.append('Int')
paramnames.append('M^0')
Lframe = pd.DataFrame(L)
Lframe.columns = paramnames

n3370
n4536
n3982
n5584
n4639
n4038
n3021
n1309


In [307]:
L[-2,:]

array([ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.])

In [308]:
print L.shape
print C.shape

(452, 12)
(452, 452)


In [309]:
param_cinv = (dot(L.T, dot(Cinv, L)))

In [310]:
LtCinv = dot(L.T, Cinv)
LtCinvY = dot(LtCinv, y)

In [311]:
bestfit = solve(param_cinv, LtCinvY)

In [312]:
bestfit

array([  2.71862951,   1.55228358,   2.3165599 ,   2.3124229 ,
         2.27572085,   2.22206887,   2.874463  ,   3.12656555,
        -2.96133472,  -0.240236  ,  26.2523365 ,  10.24997598])

In [414]:
error = diag(inv(param_cinv))
error = (1/diag(param_cinv))
error = sqrt(error)

In [415]:
print paramnames

['n3370', 'n4536', 'n3982', 'n5584', 'n4639', 'n4038', 'n3021', 'n1309', 'Per', 'Met', 'Int', 'M^0']


In [416]:
 zip(paramnames, zip(bestfit, error))

[('n3370', (2.7186295059725305, 0.029420946325746652)),
 ('n4536', (1.5522835822373897, 0.032494353688442272)),
 ('n3982', (2.3165599011298337, 0.043417396512919204)),
 ('n5584', (2.312422903614455, 0.029580579432308685)),
 ('n4639', (2.2757208494597569, 0.050107070019617228)),
 ('n4038', (2.2220688720140878, 0.055833926206266871)),
 ('n3021', (2.8744630036744225, 0.067940683031092269)),
 ('n1309', (3.1265655519976709, 0.045529284401057094)),
 ('Per', (-2.9613347179195753, 0.0079199605114396096)),
 ('Met', (-0.24023600451392926, 0.071994642481471388)),
 ('Int', (26.252336504869188, 0.012850698448006728)),
 ('M^0', (10.249975978399275, 0.039511420327395685))]

In [417]:
m04258 = [bestfit[-1], error[-1]]
a_nu = [0.698, 0.00225]
mu_geometric = [7.60, 0.32]
H_0 = [10**(a_nu[0] + 5 + 0.2 * (m04258[0] - 5*log10(mu_geometric[0]) - 25))]
H_0.append(
            H_0[0] * sqrt(
        (a_nu[1] * log(10))**2 
        + (log(10)/5 * m04258[1])**2
        + (mu_geometric[1]/mu_geometric[0])**2             
    )     )

In [418]:
print H_0

[73.651501222569479, 3.3997782428024248]


[73.651501222569479, 3.1260430605242928]
